Source :
* https://github.com/roboflow/notebooks/blob/main/notebooks/how-to-track-and-count-vehicles-with-yolov8.ipynb
* https://youtu.be/j10j8IuKSBI?feature=shared

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Mon Dec 11 16:31:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os
HOME = os.getcwd()
print(HOME)

/content


1. Load Test dataset

* Upload dataset under the /content/test directory.

In [4]:
SOURCE_VIDEO_PATH = "/content/test/scene1_01.mp4"
VIDEO_NAME = "scene1_01.mp4"

2. Intall Yolo8

In [5]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.226 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.0/78.2 GB disk)


3. Install Byte Track

In [6]:
%cd /content
!git clone https://github.com/ifzhang/ByteTrack.git
%cd /content/ByteTrack

# workaround related to https://github.com/roboflow/notebooks/issues/80
!sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt

!pip3 install -q -r requirements.txt
!python3 setup.py -q develop
!pip install -q cython_bbox
!pip install -q onemetric
!pip install -q loguru lap thop

from IPython import display
display.clear_output()


import sys
sys.path.append(f"/content/ByteTrack")


import yolox
print("yolox.__version__:", yolox.__version__)

yolox.__version__: 0.1.0


In [7]:
from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass


@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 30
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False

4. Install Roboflow Supervision

In [8]:
!pip install supervision==0.1.0


from IPython import display
display.clear_output()


import supervision
print("supervision.__version__:", supervision.__version__)

supervision.__version__: 0.1.0


In [9]:
from supervision.draw.color import ColorPalette
from supervision.geometry.dataclasses import Point
from supervision.video.dataclasses import VideoInfo
from supervision.video.source import get_video_frames_generator
from supervision.video.sink import VideoSink
from supervision.notebook.utils import show_frame_in_notebook
from supervision.tools.detections import Detections, BoxAnnotator
from supervision.tools.line_counter import LineCounter, LineCounterAnnotator

5. Tracking Utils

In [10]:
from typing import List

import numpy as np


# converts Detections into format that can be consumed by match_detections_with_tracks function
def detections2boxes(detections: Detections) -> np.ndarray:
    return np.hstack((
        detections.xyxy,
        detections.confidence[:, np.newaxis]
    ))


# converts List[STrack] into format that can be consumed by match_detections_with_tracks function
def tracks2boxes(tracks: List[STrack]) -> np.ndarray:
    return np.array([
        track.tlbr
        for track
        in tracks
    ], dtype=float)


# matches our bounding boxes with predictions
def match_detections_with_tracks(
    detections: Detections,
    tracks: List[STrack]
) -> Detections:
    if not np.any(detections.xyxy) or len(tracks) == 0:
        return np.empty((0,))

    tracks_boxes = tracks2boxes(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detections.xyxy)
    track2detection = np.argmax(iou, axis=1)

    tracker_ids = [None] * len(detections)

    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            tracker_ids[detection_index] = tracks[tracker_index].track_id

    return tracker_ids

6. Load pre-trained YOLOv8 model

In [11]:
# settings
MODEL = "yolov8x.pt"

In [12]:
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

100%|██████████| 131M/131M [00:02<00:00, 67.1MB/s]


YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs


7. Predict and annotate whole video

In [13]:
# settings
LINE_START = Point(50, 1500)
LINE_END = Point(3840-50, 1500)

TARGET_VIDEO_PATH = f"/content/test/scene1_01-result.mp4"

In [14]:
VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

VideoInfo(width=1280, height=960, fps=30, total_frames=1562)

In [15]:
# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names
# class_ids of interest - car, motorcycle, bus and truck
CLASS_ID = [2, 3, 5, 7]

In [16]:
import cv2
import time

* Remove "result.txt" from /content/ByteTrack

In [17]:
from tqdm.notebook import tqdm

# create BYTETracker instance
byte_tracker = BYTETracker(BYTETrackerArgs())
# create VideoInfo instance
video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
# create frame generator
generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
# create LineCounter instance
line_counter = LineCounter(start=LINE_START, end=LINE_END)
# create instance of BoxAnnotator and LineCounterAnnotator
box_annotator = BoxAnnotator(color=ColorPalette(), thickness=4, text_thickness=4, text_scale=2)
line_annotator = LineCounterAnnotator(thickness=4, text_thickness=4, text_scale=2)

area_2 = [(400,0), (1160, 0), (400, 270), (1160, 270)]
area_1 = [(0,622), (1250,622), (0, 1000), (1370, 1000)]

vehicles_entering = {}
vehicles_elapsed_time = {}
written_obj=[]
count = 0

# open target video file
with VideoSink(TARGET_VIDEO_PATH, video_info) as sink:

    with open('speed_result.txt', 'w+') as file1:
        file1.write("<test_video_name> <frame_no> <obj_id> <xmin> <ymin>  <xmax> <ymax>  <speed> <confidence>")
        # loop over video frames
        for frame in tqdm(generator, total=video_info.total_frames):
            #reduce frame
            #count += 1
            #if count % 3 != 0 :
            #    continue

            results = model(frame)
            detections = Detections(
                xyxy=results[0].boxes.xyxy.cpu().numpy(),
                confidence=results[0].boxes.conf.cpu().numpy(),
                class_id=results[0].boxes.cls.cpu().numpy().astype(int)
            )
            # filtering out detections with unwanted classes
            mask = np.array([class_id in CLASS_ID for class_id in detections.class_id], dtype=bool)
            detections.filter(mask=mask, inplace=True)
            # tracking detections
            tracks = byte_tracker.update(
                output_results=detections2boxes(detections=detections),
                img_info=frame.shape,
                img_size=frame.shape
            )
            tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)
            detections.tracker_id = np.array(tracker_id)
            # filtering out detections without trackers
            mask = np.array([tracker_id is not None for tracker_id in detections.tracker_id], dtype=bool)
            detections.filter(mask=mask, inplace=True)
            # format custom labels
            labels = [
                f"#{tracker_id} {CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
                for _, confidence, class_id, tracker_id
                in detections
            ]
            # updating line counter
            line_counter.update(detections=detections)
            # annotate and display frame
            frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)
            line_annotator.annotate(frame=frame, line_counter=line_counter)
            sink.write_frame(frame)
            print(detections.xyxy) # position
            print(detections.confidence) #confidence
            print(detections.tracker_id) #object id
            print(count) #frame.no

            #SpeedCheck
            for object_id, box, confidence in zip(detections.tracker_id, detections.xyxy, detections.confidence) :
                (x , y, x2, y2) = box
                cx = int((x+x2)/2)
                cy = int((y+y2)/2)
                first_capture = cv2.pointPolygonTest(np.array(area_1, np.int32), (int(cx), int(cy)), False)
                #The moment - vehicle enter first
                if first_capture >= 0 :
                    vehicles_entering[object_id] = time.time()
                print(vehicles_entering)
                #The moment - vehicle captured twice
                if object_id in vehicles_entering :
                    second_capture = cv2.pointPolygonTest(np.array(area_2, np.int32), (int(cx), int(cy)), False)
                    if second_capture >= 0 :
                        elapsed_time = time.time() - vehicles_entering[object_id]
                        if object_id not in vehicles_elapsed_time :
                            vehicles_elapsed_time[object_id] = elapsed_time
                        if object_id in vehicles_elapsed_time :
                            elapsed_time = vehicles_elapsed_time[object_id]
                        #Calc average speed
                        distance = 40 #meters
                        a_speed_ms = distance / elapsed_time
                        a_speed_kh = a_speed_ms * 3.6
                        print("a_speed_kh = " + str(a_speed_kh))
                        #Record
                        #<test_video_name> <frame_no> <obj_id> <xmin> <ymin>  <xmax> <ymax>  <speed> <confidence>
                        if str(object_id) not in written_obj :
                            written_obj.append(str(object_id))
                            file1.write("\n"+ str(VIDEO_NAME) +" " + str(count)+" <" + str(object_id) +"> " + str(box) + " " + str(a_speed_kh) + " " + str(confidence))
        file1.close()
            #%matplotlib inline
            #show_frame_in_notebook(frame, (16, 16))


  0%|          | 0/1562 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
{1: 1702313418.800795, 11: 1702313268.3577495, 21: 1702313269.7380035, 28: 1702313272.223234, 40: 1702313274.8334346, 45: 1702313275.6323779, 47: 1702313279.1185527, 49: 1702313276.1579633, 55: 1702313277.5252771, 52: 1702313278.2739413, 66: 1702313282.4320982, 68: 1702313282.2259135, 73: 1702313284.0193677, 75: 1702313284.9816945, 82: 1702313287.5556107, 84: 1702313291.3382292, 85: 1702313289.4672394, 97: 1702313294.731508, 101: 1702313294.8336396, 104: 1702313299.4803114, 112: 1702313301.8288968, 117: 1702313307.8592937, 116: 1702313306.377968, 119: 1702313307.2362728, 120: 1702313309.5406656, 121: 1702313311.6015465, 124: 1702313314.7019835, 126: 1702313314.8120115, 128: 1702313318.6222186, 132: 1702313319.963745, 134: 1702313322.5328162, 139: 1702313325.5311568, 140: 1702313329.9609735, 141: 1702313329.959631, 143: 1702313330.4661305, 147: 1702313332.8319154, 150: 1702313336.7917237, 151: 1702313335.83621, 155: 1702313335.9377055, 

In [18]:
!pwd

/content/ByteTrack


### After execution of above codes, you can check "result.txt" file in the /content/ByteTrack location.


###Result.txt

\<test_video_name> \<frame_no> \<obj_id> \<xmin> \<ymin>  \<xmax> \<ymax>  \<speed> \<confidence>        
scene1_01.mp4 63 <3394> [     591.52      222.15      644.78      313.61] 68.64520469912617 0.7455659              
scene1_01.mp4 93 <3403> [     435.53      235.25      489.21      296.83] 65.22137198829691 0.8589536            
scene1_01.mp4 168 <3414> [     580.23      154.25      659.12      377.89] 58.21693261482494 0.8928519           
scene1_01.mp4 216 <3443> [     656.02      240.17      705.03      298.17] 53.83304814529135 0.793149           
scene1_01.mp4 267 <3452> [     527.69      214.64      581.65      308.39] 71.54305484385206 0.6881396           
scene1_01.mp4 297 <3430> [     651.37      178.08      736.91      356.74] 60.30689007323339 0.93648076          ....

In [18]:
!cp "/content/ByteTrack/speed_result.txt" "/content/drive/MyDrive/Juyoung Lee_/Model evaluation on test datasets video/"

In [19]:
!cp -r "/content/ByteTrack/" "/content/drive/MyDrive/Juyoung Lee_/VehicleSpeedDetection/"

In [20]:
!cp -r "/content/test/" "/content/drive/MyDrive/Juyoung Lee_/VehicleSpeedDetection/"